## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, trim, lower, regexp_replace, current_timestamp
from pyspark.sql.types import StringType
import os
import re

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Mapping_Post_Entity_ORG")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session đã được khởi tạo với Nessie catalog!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

## 2. Định nghĩa Organization Mapping Dictionary

Dictionary này chứa các quy luật mapping trường/tổ chức, tương tự như `normalize_orgs.py`

In [ ]:
# Dictionary mapping các trường đại học/tổ chức - TỐI ƯU (mỗi nhóm 1 dòng)
org_mapping = {
    'đại học quốc gia hà nội': ['đhqghn', 'đại học quốc gia hà nội', 'đh quốc gia hn', 'đh công nghệ - đhqghn', 'hus', 'vnu', 'đại học quốc gia thành phố hồ chí minh', 'đhqg tp.hcm', 'đhqg tphcm', 'đại học quốc gia tp.hcm', 'quốc gia tp. hcm', 'đại học hà nội', 'đh công nghệ', 'uet', 'đhqg-hcm', 'hanu', 'vnu-sis', 'vnu-ifi', 'đại học qt-đhqghcm', 'đhqg tp. hcm', 'đại học quốc gia tphcm', 'vju-vnu'],
    'đại học bách khoa hà nội': ['bách khoa hà nội', 'đhbk hà nội', 'hust', 'bkhn', 'đại học bách khoa', 'đh bách khoa hà nội', 'bách khoa', 'đh bách khoa', 'bku'],
    'đại học bách khoa tp.hcm': ['đại học bách khoa thành phố hồ chí minh', 'đại học bách khoa tp.hcm', 'bách khoa tp. hcm', 'đh bách khoa tphcm', 'bách khoa tphcm', 'bách khoa sài gòn', 'hubt'],
    'đại học kinh tế quốc dân': ['đại học kinh tế quốc dân', 'neu', 'kinh tế quốc dân', 'trường kinh tế quốc dân', 'ueb'],
    'đại học sư phạm hà nội': ['đại học sư phạm hà nội', 'sư phạm hà nội', 'đhsp hà nội', 'hnue', 'đh sư phạm hà nội'],
    'đại học sư phạm tp.hcm': ['đại học sư phạm tp.hcm', 'sư phạm tphcm', 'hcmue', 'đại học sư phạm thành phố hồ chí minh', 'sư phạm', 'đh sư phạm tphcm', 'sư phạm tp. hcm', 'đhsp huế'],
    'đại học y hà nội': ['đại học y hà nội', 'y hà nội', 'đh y hn', 'dhyhn', 'hmu', 'đại học y', 'trường y', 'trường đại học y', 'đại học chính quy y', 'đh y'],
    'đại học y dược tp.hcm': ['đại học y dược tp.hcm', 'y dược tphcm', 'uth', 'đại học y dược thành phố hồ chí minh', 'đh y dược tp. hcm', 'y dược tp. hcm', 'yds'],
    'đại học cần thơ': ['đại học cần thơ', 'trường đại học cần thơ', 'ctu', 'phân hiệu trường đại học cần thơ', 'đh cần thơ', 'ctut'],
    'đại học y dược cần thơ': ['đại học y dược cần thơ', 'y dược cần thơ'],
    'đại học luật hà nội': ['đại học luật hà nội', 'luật hà nội', 'hlu', 'luật hn', 'sol'],
    'đại học luật tp.hcm': ['đại học luật tp.hcm', 'luật tphcm', 'đh luật tp. hcm', 'đại học luật thành phố hồ chí minh', 'trường đại học luật tp hcm'],
    'đại học ngoại thương': ['đại học ngoại thương', 'ngoại thương', 'ftu', 'trường đại học ngoại thương cơ sở 2', 'fbu'],
    'đại học văn lang': ['đại học văn lang', 'văn lang', 'đh văn lang', 'vlu'],
    'đại học hoa sen': ['đại học hoa sen', 'hoa sen', 'trường đại học hoa sen', 'hsu'],
    'đại học kinh tế tp.hcm': ['đại học kinh tế tp.hcm', 'ueh', 'kinh tế tphcm', 'trường đại học kinh tế', 'đại học kinh tế thành phố hồ chí minh', 'uehmekong', 'uehers'],
    'đại học mở tp.hcm': ['đại học mở tp.hcm', 'ou', 'open university', 'đại học mở thành phố hồ chí minh', 'đh mở tp.hcm', 'đại học mở', 'đh mở', 'đh mở tphcm', 'trường đại học mở tp. hồ chí minh', 'trường đại học mở tphcm', 'trường đh mở', 'hcmcou', 'đại học mở tp. hcm'],
    'đại học fpt': ['đại học fpt', 'đh fpt', 'fpt university', 'fpt polytechnic', 'fpt aptech', 'fpt polytechnic đồng nai', 'cao đẳng fpt', 'đại học fpt truyền thông đa phương tiện', 'fpt long châu', 'tập đoàn fpt'],
    'đại học phenikaa': ['đại học phenikaa', 'phenikaa', 'phenikaa university', 'phenikaauni', 'pka'],
    'đại học xây dựng hà nội': ['đại học xây dựng hà nội', 'xây dựng hà nội', 'trường đại học xây dựng hà nội', 'huce', 'đại học xây dựng'],
    'đại học giao thông vận tải': ['đại học giao thông vận tải', 'giao thông vận tải', 'trường đại học giao thông vận tải', 'ut', 'đh giao thông vận tải cơ sở 2', 'giao thông vận tải tphcm', 'gtvt', 'utc', 'utc 2', 'utc2', 'trường utc'],
    'đại học thủy lợi': ['đại học thủy lợi', 'thủy lợi', 'trường thuỷ lợi'],
    'đại học hàng hải': ['đại học hàng hải', 'hàng hải', 'trường đại học hàng hải', 'cao đẳng hàng hải', 'vimaru'],
    'đại học công nghiệp thực phẩm tp.hcm': ['đại học công nghiệp thực phẩm thành phố hồ chí minh', 'công nghiệp thực phẩm tphcm', 'cao đẳng công nghiệp', 'hcmufa'],
    'đại học điện lực': ['đại học điện lực', 'điện lực', 'đh điện lực'],
    'đại học mỹ thuật tp.hcm': ['đại học mỹ thuật tp. hồ chí minh', 'mỹ thuật tphcm', 'mỹ thuật việt nam', 'đại học mỹ thuật'],
    'học viện ngoại giao': ['học viện ngoại giao', 'ngoại giao'],
    'học viện cảnh sát nhân dân': ['học viện cảnh sát nhân dân', 'cảnh sát nhân dân', 'công an nhân dân', 'cảnh sát', 'công an'],
    'học viện chính sách và phát triển': ['học viện chính sách và phát triển', 'apd'],
    'đại học sư phạm thái nguyên': ['đại học sư phạm thái nguyên', 'sư phạm thái nguyên', 'tnue'],
    'đại học thái nguyên': ['đại học thái nguyên', 'trường công nghiệp kỹ thuật thái nguyên', 'tnu', 'tnut', 'đh thái nguyên', 'tnu-s', 'nttu'],
    'đại học quy nhơn': ['đại học quy nhơn', 'quy nhơn', 'đh quy nhơn', 'qnu'],
    'đại học huế': ['đại học huế', 'đh sư phạm huế', 'trường cao đẳng y tế huế', 'đh huế', 'đại học khoa học huế', 'hue'],
    'đại học đà nẵng': ['đại học đà nẵng', 'y dược đại học đà nẵng', 'trường đại học thể dục thể thao đà nẵng', 'dnu', 'dut', 'đà nẵng', 'due'],
    'đại học nha trang': ['đại học nha trang', 'đh nha trang', 'trường đại học ntn'],
    'đại học tây nguyên': ['đại học tây nguyên', 'trường đại học tây nguyên', 'đại học tây bắc'],
    'đại học công nghệ thông tin tp.hcm': ['đại học công nghệ thông tin tphcm', 'đại học công nghệ thông tin', 'trường đại học công nghệ thông tin', 'đh cntt', 'cntt', 'uit', 'thông tin', 'huis'],
    'đại học bưu chính viễn thông': ['ptit', 'bưu chính viễn thông', 'học viện bưu chính viễn thông', 'học viện công nghệ bưu chính viễn thông'],
    'đại học công nghiệp hà nội': ['đại học công nghiệp hà nội', 'công nghiệp hà nội', 'haui', 'hau'],
    'đại học công nghiệp tp.hcm': ['đại học công nghiệp tp.hcm', 'đại học công nghiệp thành phố hồ chí minh', 'iuh', 'huit', 'công nghiệp tphcm'],
    'đại học quốc tế': ['đại học quốc tế sài gòn', 'đại học quốc tế', 'đại học anh quốc việt nam', 'sài gòn', 'trung cấp sài gòn', 'đh sài gòn', 'hcmiu'],
    'đại học greenwich việt nam': ['đại học greenwich việt nam', 'đh greenwich việt nam', 'greenwich', 'ich'],
    'đại học thủ dầu một': ['đại học thủ dầu một', 'thủ dầu một', 'tdmu', 'tdc'],
    'đại học thành đô': ['đại học thành đô', 'trường đại học thành đô', 'thành đô', 'đại học thành đông'],
    'đại học đại nam': ['đại học đại nam', 'đại nam'],
    'đại học hạ long': ['đại học hạ long', 'trường đại học hạ long'],
    'đại học phan châu trinh': ['đại học phan châu trinh', 'phan châu trinh'],
    'đại học tân trào': ['đại học tân trào', 'đh tân trào'],
    'đại học điều dưỡng nam định': ['đại học điều dưỡng nam định', 'trường đại học điều dưỡng nam định'],
    'đại học y tế công cộng': ['đại học y tế công cộng', 'trường đại học y tế công cộng'],
    'học viện âm nhạc': ['học viện âm nhạc', 'viện âm nhạc', 'đại học nhạc viện', 'nhạc viện tp. hcm', 'học viện âm nhạc quốc gia việt nam'],
    'đại học văn hóa tp.hcm': ['đại học văn hóa tp.hcm', 'văn hoá tp.hcm', 'đại học văn hóa thành phố hồ chí minh', 'đại học văn hoá hà nội', 'hcmuc'],
    'đại học việt - hàn': ['trường đại học công nghệ thông tin và truyền thông việt-hàn'],
    'đại học khoa học tự nhiên tp.hcm': ['đại học khoa học tự nhiên tphcm', 'đh khoa học tự nhiên tphcm', 'đại học khoa học tự nhiên thành phố hồ chí minh', 'đại học khoa học tự nhiên', 'hcmus', 'khoa học tự nhiên', 'đh khtn', 'khtn tp. hcm', 'hce'],
    'đại học khoa học xã hội và nhân văn': ['đại học khoa học xã hội và nhân văn', 'ussh', 'đh khoa học xã hội và nhân văn', 'đh khxh & nv', 'đh khxh & nv hn', 'đh khxh & nv tphcm', 'khoa học xã hội và nhân văn', 'khxhnv', 'nhân văn', 'đh khxh & nv tp. hcm', 'khxh & nv'],
    'đại học khoa học & công nghệ hà nội': ['đại học kh & cn hà nội', 'usth', 'đại học usth', 'đại học khoa học & công nghệ hà nội'],
    'đại học mỏ - địa chất': ['đại học mỏ - địa chất', 'đh mỏ - địa chất', 'mỏ - địa chất', 'đại học mỏ', 'humg', 'trường đại học h chuyên mỏ'],
    'đại học công nghiệp': ['đại học công nghiệp', 'đh công nghiệp'],
    'đại học công thương': ['đại học công thương', 'đh công thương', 'cao đẳng công thương', 'công thương tp. hcm', 'công thương thành phố hồ chí minh', 'công thương', 'bộ công thương'],
    'đại học thương mại': ['đại học thương mại', 'thương mại', 'tmu', 'trường đại học thương mại', 'bộ thương mại'],
    'đại học ngoại ngữ': ['đại học ngoại ngữ', 'ulis', 'ngoại ngữ', 'đại học ngoại ngữ - tin học thành phố hồ chí minh', 'ufls', 'daihocngoaingudanang'],
    'đại học sư phạm kỹ thuật tp.hcm': ['đại học sư phạm kỹ thuật tp.hcm', 'sư phạm kỹ thuật tphcm', 'hcmute', 'sư phạm kỹ thuật', 'spkt hcm'],
    'đại học nông lâm tp.hcm': ['đại học nông lâm', 'nông lâm', 'đại học nông lâm thành phố hồ chí minh', 'nông lâm huế', 'đại học nông lâm huế'],
    'đại học ngân hàng': ['đại học ngân hàng thành phố hồ chí minh', 'ngân hàng tphcm', 'hub', 'đại học ngân hàng', 'ngân hàng a'],
    'đại học tài chính - marketing': ['đại học tài chính - marketing', 'tài chính - marketing', 'ufm'],
    'đại học tài nguyên và môi trường': ['đại học tài nguyên và môi trường', 'tài nguyên và môi trường', 'hunre', 'tài nguyên & môi trường tp. hcm', 'tài nguyên môi trường'],
    'đại học y dược thái bình': ['y dược thái bình', 'đại học y dược thái bình', 'y thái bình'],
    'đại học y hải phòng': ['y hải phòng', 'đại học y hải phòng', 'hpu 2', 'hpu2'],
    'đại học dược hà nội': ['dược hà nội', 'đại học dược hà nội', 'trường dược'],
    'cao đẳng y tế': ['cao đẳng y tế', 'trung cấp y dược sài gòn', 'trường trung cấp y dược sài gòn', 'cao đẳng y', 'trường cao đẳng y', 'cao đẳng y dược pasteur'],
    'cao đẳng cơ điện xây dựng': ['cao đẳng cơ điện và xây dựng bắc ninh'],
    'cao đẳng phát thanh truyền hình': ['cao đẳng phát thanh truyền hình ii', 'cao đẳng phát thanh truyền hình'],
    'cao đẳng lý tự trọng': ['trường cao đẳng lý tự trọng tphcm', 'lý tự trọng'],
    'cao đẳng hà nội': ['cao đẳng hà nội', 'hnc'],
    'cao đẳng bách khoa hà nội': ['cao đẳng công nghệ bách khoa hà nội', 'cao đẳng bách khoa hà nội', 'hpc'],
    'cao đẳng công nghệ thông tin tp.hcm': ['cao đẳng công nghệ thông tin tp.hcm', 'itc'],
    'cao đẳng công nghệ cao hà nội': ['cao đẳng công nghệ cao hà nội', 'hht'],
    'cao đẳng công thương hải phòng': ['cao đẳng công thương hải phòng', 'hpcom'],
    'cao đẳng kỹ thuật công nghệ': ['cao đẳng kỹ thuật công nghệ ctech', 'ctech'],
    'cao đẳng nghề aspace': ['cao đẳng nghề aspace', 'aspace'],
    'cao đẳng cơ điện thủy lợi': ['cao đẳng cơ điện và thủy lợi', 'cmet'],
    'cao đẳng văn hóa nghệ thuật tp.hcm': ['cao đẳng văn hóa nghệ thuật tp.hcm', 'vhs'],
    'cao đẳng công nghiệp dệt may hà nội': ['cao đẳng công nghiệp dệt may thời trang hà nội', 'hitc'],
    'học viện hàng không việt nam': ['học viện hàng không việt nam', 'học viện hàng không', 'hàng không việt nam', 'vaa'],
    'học viện quân y': ['học viện quân y', 'quân y', 'hv quân y', 'mta'],
    'học viện hành chính quốc gia': ['học viện hành chính quốc gia', 'hành chính quốc gia', 'napa'],
    'học viện thanh thiếu niên': ['học viện thanh thiếu niên', 'thanh thiếu niên'],
    'học viện hậu cần': ['học viện hậu cần', 'hậu cần'],
    'học viện tư pháp': ['học viện tư pháp'],
    'học viện biên phòng': ['học viện biên phòng', 'bph'],
    'học viện ngân hàng': ['học viện ngân hàng', 'ngân hàng', 'banking academy', 'bav', 'hvnh'],
    'học viện phụ nữ việt nam': ['học viện phụ nữ việt nam', 'phụ nữ việt nam', 'học viện phụ nữ', 'vwa'],
    'trường sĩ quan không quân': ['trường sĩ quan không quân', 'sĩ quan không quân', 'không quân'],
    'đại học hoa lư': ['đại học hoa lư', 'hoa lư'],
    'đại học kiên giang': ['đại học kiên giang', 'kiên giang', 'trường đại học kiên giang'],
    'đại học phú xuân': ['đại học phú xuân', 'phú xuân', 'trường đại học phú xuân'],
    'đại học cửu long': ['đại học cửu long', 'cửu long', 'trường đại học cửu long'],
    'đại học nghĩa thủ': ['đại học nghĩa thủ', 'nghĩa thủ'],
    'đại học việt mỹ': ['việt mỹ', 'đại học việt mỹ', 'vmu'],
    'đại học lao động xã hội': ['lao động xã hội', 'đại học lao động xã hội'],
    'đại học nghệ an': ['đại học nghệ an', 'đh nghệ an', 'trường đại học nghệ an'],
    'đại học quảng nam': ['đại học quảng nam'],
    'đại học phú yên': ['đại học phú yên'],
    'đại học đồng tháp': ['đại học đồng tháp', 'trường đại học đồng tháp', 'trường đh đồng tháp', 'trường đại học đồng thápcho'],
    'đại học phạm văn đồng': ['đại học phạm văn đồng', 'trường đại học phạm văn đồng'],
    'đại học võ trường toản': ['đại học võ trường toản'],
    'đại học bình dương': ['đại học bình dương', 'trường đại học bình dương bdu'],
    'đại học nguyên trí': ['đại học nguyên trí'],
    'đại học lương thế vinh': ['đại học lương thế vinh', 'đh lương thế vinh'],
    'đại học hùng vương': ['đại học hùng vương', 'trường đại học hùng vương'],
    'đại học lạc hồng': ['đại học lạc hồng'],
    'đại học tân tạo': ['đại học tân tạo', 'trường đại học tân tạo', 'đh tân tạo', 'trường đại học tân hoa'],
    'đại học hoà bình': ['đại học hoà bình', 'đh hoà bình'],
    'đại học sân khấu điện ảnh': ['đại học sân khấu điện ảnh', 'trường đại học sân khấu điện ảnh', 'sân khấu điện ảnh', 'trường sân khấu điện ảnh'],
    'đại học chân lý': ['đại học chân lý'],
    'đại học ngoại ngữ - tin học tp.hcm': ['đại học ngoại ngữ - tin học tp.hcm', 'huflit'],
    'đại học southern california': ['university of southern california', 'usc'],
    'đại học chonnam': ['đại học quốc gia chonnam', 'chonnam'],
    'đại học kimpo': ['đại học kimpo', 'kimpo'],
    'đại học paichai': ['đại học paichai', 'paichai'],
    'đại học sun sinip': ['đại học sun sinip', 'sun sinip'],
    'đại học shinansan': ['trường shinansan', 'shinansan'],
    'đại học bắc kinh': ['đại học bắc kinh', 'bắc kinh'],
    'đại học trùng khánh': ['đại học trùng khánh', 'trùng khánh'],
    'đại học đài loan': ['đại học đài loan', 'đài loan', 'đại học đông nam - đài loan', 'trường đại học đông nam - đài loan', 'cao hùng'],
    'đại học trung quốc': ['đại học trung quốc', 'an huy trung quốc', 'qufu normal university', 'trường quảng ngôn'],
    'loughborough university': ['loughborough university', 'loughborough'],
    'aston university': ['aston university', 'aston'],
    'texas state university': ['texas state university', 'texas state'],
    'postech': ['postech', 'đại học khoa học công nghệ pohang'],
    'kaist': ['kaist', 'trường kaist', 'korea advanced institute of science and technology'],
    'keio university': ['keio university', 'trường keio'],
    'snu': ['snu', 'trường snu', 'đại học quốc gia seoul'],
    'harvard university': ['harvard university', 'harvard'],
    'oxford university': ['oxford university', 'đại học oxford'],
    'british council': ['british council', 'hội đồng anh', 'bc'],
    'aptech ấn độ': ['aptech ấn độ', 'aptech'],
    'monash university': ['monash univeristy', 'monash university', 'monash'],
    'university of melbourne': ['melbourne polytechnic australia', 'melbourne polytechnic việt nam', 'melbourne polytechnic'],
    'la trobe university': ['đại học la trobe'],
    'flinders university': ['flinders university'],
    'southern cross university': ['southern cross university'],
    'university of technology sydney': ['uts', 'trường uts'],
    'queensland university of technology': ['đại học qut'],
    'swansea university': ['swansea university'],
    'goldsmiths university': ['goldsmiths university', 'goldsmiths'],
    'university of bradford': ['university of bradford', 'bradford'],
    'leeds beckett university': ['leeds beckett university', 'leeds beckett'],
    'university of law': ['university of law', 'ulaw'],
    'anglia ruskin university': ['anglia ruskin university'],
    'đại học new brunswick': ['đại học new brunswick'],
    'đại học kỹ thuật y tế hải dương': ['kỹ thuật y tế hải dương', 'đh kỹ thuật y tế hải dương'],
    'bộ giáo dục và đào tạo': ['bộ giáo dục và đào tạo', 'bộ gd & đt', 'giáo dục', 'đại học giáo dục', 'bộ giáo dục', 'bộ gdđt', 'bộ gd-đt', 'bộ'],
    'sở giáo dục và đào tạo': ['sở gd & đt', 'sở giáo dục và đào tạo', 'sở gd & đt tphcm'],
    'bộ quốc phòng': ['bộ quốc phòng', 'công binh', 'hoàng gia', 'học viện hoàng gia', 'đặc công', 'tăng - thiết giáp'],
    'bộ y tế': ['bộ y tế', 'bạch mai'],
    'bộ công an': ['bộ công an'],
    'bộ tài nguyên môi trường': ['bộ tnmt'],
    'bộ khoa học công nghệ': ['bộ kh & cn'],
    'bộ nông nghiệp': ['bộ nn & ptnt'],
    'chính phủ việt nam': ['chính phủ', 'chính phủ việt nam'],
    'đại học nguyễn tất thành': ['đại học nguyễn tất thành', 'nguyễn tất thành', 'ntt', 'đh nguyễn tất thành', 'trung cấp nguyễn tất thành'],
    'đại học vinh': ['đại học vinh', 'đh vinh', 'trường đh vinh', 'vinh university', 'dhv'],
    'đại học duy tân': ['đại học duy tân', 'duy tân', 'dtu'],
    'đại học đông á': ['đại học đông á', 'đông á', 'đại học đông á đà nẵng', 'dav', 'uah'],
    'đại học rmit': ['rmit', 'đại học rmit'],
    'đại học vinuni': ['vinuni', 'đại học vinuni', 'vin university'],
    'đại học an giang': ['đại học an giang', 'an giang', 'agu'],
    'đại học sài gòn': ['đại học sài gòn', 'sgu'],
    'học viện báo chí và tuyên truyền': ['học viện báo chí và tuyên truyền', 'học viện báo chí & tuyên truyền', 'báo chí và tuyên truyền', 'ajc'],
    'học viện tài chính': ['học viện tài chính', 'tài chính', 'tài chính – marketing'],
    'học viện nông nghiệp việt nam': ['học viện nông nghiệp việt nam', 'nông nghiệp việt nam', 'nông nghiệp', 'vnua', 'đại học nông nghiệp trung quốc'],
    'đại học hồng bàng': ['đại học hồng bàng', 'hồng bàng', 'đại học quốc tế hồng bàng', 'quốc tế hồng bàng'],
    'đại học kiến trúc tp.hcm': ['đại học kiến trúc', 'kiến trúc', 'đại học kiến trúc tphcm', 'đại học kiến trúc tp. hcm', 'kiến trúc thành phố hồ chí minh'],
    'đại học tôn đức thắng': ['đại học tôn đức thắng', 'tôn đức thắng', 'tdtu', 'tdt'],
    'đại học hutech': ['hutech', 'đại học hutech'],
    'đại học thủ đô hà nội': ['đại học thủ đô hà nội', 'thủ đô hà nội', 'hnmu'],
    'đại học gdu': ['gdu', 'đại học gia định', 'đh gia định'],
    'đại học văn hiến': ['đại học văn hiến', 'vhu'],
    'đại học y khoa phạm ngọc thạch': ['đại học y khoa phạm ngọc thạch', 'y khoa phạm ngọc thạch', 'đh y khoa phạm ngọc thạch'],
    'đại học kinh tế - luật': ['đại học kinh tế - luật', 'đh kinh tế - luật', 'đh kinh tế luật', 'kinh tế - luật', 'uel'],
    'đại học nội vụ hà nội': ['đại học nội vụ hà nội', 'nội vụ hà nội', 'uneti'],
    'đại học thể dục thể thao': ['đại học thể dục thể thao', 'thể dục thể thao', 'đại học thể dục thể thao đà nẵng'],
    'đại học thanh đảo': ['đại học thanh đảo', 'thanh đảo'],
    'đại học chung-ang': ['đại học chung-ang', 'chung ang', 'chung-ang'],
    'đại học konkuk': ['konkuk', 'đại học konkuk'],
    'đại học yonsei': ['yonsei', 'đại học yonsei'],
    'đại học i-shou': ['i-shou university', 'i-shou', 'ishou', 'isu'],
    'đại học kinh tế huế': ['đại học kinh tế huế', 'kinh tế huế'],
    'đại học công nghiệp và thương mại': ['đại học công nghiệp và thương mại hà nội', 'công nghiệp và thương mại'],
    'đại học đồng nai': ['đại học đồng nai', 'đồng nai', 'phân hiệu đại học lâm nghiệp đồng nai'],
    'đại học trà vinh': ['đại học trà vinh', 'trà vinh'],
    'đại học công đoàn': ['đại học công đoàn', 'công đoàn'],
    'đại học kinh tế tài chính tp.hcm': ['đại học kinh tế tài chính thành phố hồ chí minh', 'kinh tế tài chính tphcm', 'uef'],
    'đại học quốc tế - đhqg hcm': ['đại học quốc tế đại học quốc gia thành phố hcm', 'đh quốc tế - đhqg tp. hcm', 'siu'],
    'học viện cán bộ tp.hcm': ['học viện cán bộ thành phố hồ chí minh', 'cán bộ tphcm'],
    'đại học tây đô': ['đại học tây đô', 'tây đô'],
    'đại học fullbright': ['đại học fullbright', 'fullbright', 'fun bright university', 'fulbright university', 'fulbright university vietnam', 'đại học fulbright việt nam'],
    'đại học vgu': ['vgu', 'đại học việt đức'],
    'đại học hoa đông': ['đại học đông hoa trung quốc', 'đông hoa'],
    'đại học nam kinh': ['đại học nam kinh', 'nam kinh'],
    'đại học thượng hải': ['đại học thượng hải', 'thượng hải', 'đại học giao thông thượng hải', 'đại học trung tín'],
    'đại học long hoa': ['đại học long hoa', 'long hoa'],
    'đại học soongeui': ['đại học nữ sinh soongeui', 'soongeui'],
    'đại học sookmyung': ['đại học nữ sinh sookmyung', 'sookmyung'],
    'đại học hongik': ['hongik', 'đại học hongik'],
    'đại học kyunghee': ['kyunghee', 'đại học kyunghee'],
    'đại học gachon': ['đại học gachon', 'gachon'],
    'đại học dong-a': ['đại học dong-a', 'dong-a', 'đại học dong eui'],
    'đại học silla': ['trường đại học silla', 'đại học silla'],
    'đại học sungkonghoe': ['đại học sungkonghoe', 'sungkonghoe'],
    'đại học nam seoul': ['nam seoul', 'đại học nam seoul'],
    'đại học toledo': ['đại học toledo', 'toledo', 'đại học toledo ohio'],
    'đại học illinois': ['đại học illinois', 'illinois'],
    'western sydney university': ['western sydney university', 'sydney'],
    'trường quân đội': ['trường quân đội', 'quân đội', 'trường quân sự', 'sĩ quan pháo binh'],
    'học viện khoa học quân sự': ['học viện khoa học quân sự', 'hv khoa học quân sự'],
    'học viện an ninh nhân dân': ['học viện an ninh nhân dân', 'an ninh nhân dân'],
    'học viện chính trị cand': ['học viện chính trị cand', 'chính trị cand'],
    'học viện toà án': ['học viện toà án', 'toà án'],
    'học viện chính trị quốc gia hồ chí minh': ['học viện chính trị quốc gia hồ chí minh', 'chính trị quốc gia hồ chí minh'],
    'đại học yersin đà lạt': ['yersin đà lạt', 'đại học yersin'],
    'đại học quang trung': ['đại học quang trung', 'quang trung'],
    'đại học phương đông': ['đại học phương đông', 'phương đông'],
    'đại học thăng long': ['trường đại học thăng long', 'đại học thăng long', 'đh thăng long'],
    'đại học quốc tế miền đông': ['đại học quốc tế miền đông'],
    'đại học quản trị và kinh doanh': ['đại học quản trị và kinh doanh'],
    'đại học quốc tế bắc hà': ['đại học quốc tế bắc hà', 'trường đại học quốc tế bắc hà'],
    'đại học kiểm sát': ['đại học kiểm sát'],
    'đại học cmc': ['đại học cmc'],
    'đại học việt nhật': ['đại học việt nhật', 'vju'],
    'đại học buv': ['buv'],
    'trường nghệ thuật': ['trường nghệ thuật'],
    'tổ chức - samsung': ['seri', 'viện nghiên cứu kinh tế samsung'],
    'tổ chức - vneid': ['vneid'],
    'tổ chức - scopus': ['scopus'],
    'tổ chức - scimago': ['scimago'],
    'tổ chức - idp': ['idp'],
    'tổ chức - erudera': ['erudera'],
    'tổ chức - việt talents': ['việt talents'],
    'tổ chức - hiệp hội bảo hiểm việt nam': ['hiệp hội bảo hiểm việt nam'],
    'công ty - vinfast': ['vinfast'],
    'công ty - samsung': ['samsung'],
    'công ty - lg': ['lg'],
    'công ty - intel': ['intel'],
    'công ty - microsoft': ['microsoft'],
    'công ty - nvidia': ['nvidia'],
    'công ty - honda': ['honda'],
    'công ty - toyota': ['toyota'],
    'công ty - nissan': ['nissan'],
    'công ty - mitsubishi': ['mitsubishi'],
    'công ty - canon': ['canon'],
    'công ty - bosch': ['bosch'],
    'công ty - unilever': ['unilever việt nam'],
    'công ty - estee lauder': ['estee lauder companies'],
    'công ty - loreal': ['l oréal group'],
    'công ty - viettel': ['viettel'],
    'công ty - vng': ['vng'],
    'công ty - shopee': ['shopee'],
    'công ty - lazada': ['lazada'],
    'công ty - tiktok': ['tiktok'],
    'công ty - vib': ['vib'],
    'công ty - mcredit': ['mcredit'],
    'công ty - vinschool': ['vinschool'],
    'công ty - careerviet': ['careerviet'],
    'công ty - du học thật': ['du học thật'],
    'công ty - appshop': ['appshop'],
    'công ty - vijalink': ['công ty vijalink'],
    'công ty - dk pharma': ['dk pharma'],
    'công ty - công ty tư vấn': ['công ty thiết kế tư vấn kỹ thuật'],
    'truyền thông - vnexpress': ['vnexpress'],
    'truyền thông - vietnamnet': ['vietnamnet'],
    'truyền thông - dân trí': ['báo dân trí'],
    'truyền thông - jtv news': ['jtv news'],
    'truyền thông - secco news': ['secco news'],
    'truyền thông - mytv': ['mytv'],
    'truyền thông - sctv': ['sctv'],
    'truyền thông - vov': ['vov'],
    'truyền thông - vietjet': ['vj việt nam', 'vj'],
    'dịch vụ giáo dục - edulife': ['edulife'],
    'dịch vụ giáo dục - fenica': ['fenica'],
    'dịch vụ giáo dục - jvnet': ['jvnet'],
    'dịch vụ giáo dục - dkedu': ['dkedu'],
}

print(f"Đã load {len(org_mapping)} nhóm trường/tổ chức với {sum(len(v) for v in org_mapping.values())} variants")
print("ĐÃ XÓA: 9 nhóm khoa, nhóm đại học chung chung (hàn quốc/úc/anh), viết tắt không rõ, cao đẳng chung, phân hiệu chung, cơ quan hành chính chung")

## 3. Định nghĩa Hàm Normalize và Mapping

In [ ]:
def normalize_text(text):
    """Chuẩn hóa tên trường - CẢI TIẾN
    1. Lowercase, trim
    2. Remove trailing punctuation + ngoặc đơn
    3. Normalize whitespace
    4. Remove unicode và các ký tự đặc biệt (GIỮ dấu gạch ngang)
    ⚠️ KHÔNG XÓA PREFIX để tránh mất thông tin khi tên quá ngắn
    """
    if not text or not isinstance(text, str):
        return ""
    
    text = text.lower().strip()
    
    # Loại bỏ các ký tự đặc biệt cuối chuỗi
    text = text.rstrip('.,;:)(!?')
    
    # Loại bỏ ngoặc đơn ở đầu/cuối và nội dung trong ngoặc
    import re
    text = re.sub(r'\([^)]*\)', '', text)  # Xóa nội dung trong ngoặc đơn
    text = text.strip('()')
    
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Loại bỏ các ký tự đặc biệt không phải chữ cái, số, dấu cách
    # GIỮ LẠI dấu gạch ngang (-) vì có các trường như "mỏ - địa chất"
    text = re.sub(r'[^\w\s\-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # ⚠️ KHÔNG XÓA PREFIX NỮA - để giữ nguyên "đại học y", "trường y", "đh mở"
    # Lý do: Khi xóa prefix, "Đại học Y" → "y" (1 ký tự) không match với variants
    
    return text


def find_org_group(normalized_name):
    """Tìm nhóm trường/tổ chức - CẢI TIẾN
    Bước 1: Exact match với key
    Bước 2: Exact match với variants
    Bước 3: Substring match (variant ≥4 chars VÀ name chứa variant)
    Bước 4: Reverse substring match (name ≥3 chars VÀ variant chứa name) - CHO VIẾT TẮT
    Bước 5: Word overlap match (≥60% words khớp)
    Trả về key (CÓ hoặc KHÔNG CÓ prefix tùy key trong dictionary)
    """
    
    # Danh sách các tổ chức hành chính (không phải trường học)
    organizations = ['bộ giáo dục và đào tạo', 'bộ quốc phòng', 'bộ công an', 'bộ y tế',
                     'sở giáo dục và đào tạo', 'bộ tài nguyên môi trường', 'bộ khoa học công nghệ',
                     'bộ nông nghiệp', 'bộ công thương', 'bộ thương mại', 'chính phủ việt nam']
    
    name = normalized_name.lower().strip()
    if not name or len(name) <= 1:
        return None
    
    # Bước 1: Exact match với key
    if name in org_mapping:
        key = name
        # Nếu key ĐÃ CÓ prefix (chứa " - ") thì trả về nguyên key
        if ' - ' in key:
            return key
        # Nếu key CHƯA CÓ prefix thì thêm prefix tùy loại
        if key in organizations:
            return f'tổ chức - {key}'
        else:
            return f'trường - {key}'
    
    # Bước 2: Exact match với variants
    for key, variants in org_mapping.items():
        if name in variants:
            # Nếu key ĐÃ CÓ prefix (chứa " - ") thì trả về nguyên key
            if ' - ' in key:
                return key
            # Nếu key CHƯA CÓ prefix thì thêm prefix tùy loại
            if key in organizations:
                return f'tổ chức - {key}'
            else:
                return f'trường - {key}'
    
    # Bước 3: Substring match - variant có ≥4 ký tự VÀ name chứa TOÀN BỘ variant
    for key, variants in org_mapping.items():
        for variant in variants:
            if len(variant) >= 4:
                if variant in name:
                    # Nếu key ĐÃ CÓ prefix (chứa " - ") thì trả về nguyên key
                    if ' - ' in key:
                        return key
                    # Nếu key CHƯA CÓ prefix thì thêm prefix tùy loại
                    if key in organizations:
                        return f'tổ chức - {key}'
                    else:
                        return f'trường - {key}'
    
    # Bước 4: Reverse substring match - name có ≥3 ký tự VÀ variant chứa name
    # DÀNH CHO CÁC TÊN VIẾT TẮT (VD: "ptit" sẽ khớp với variant "ptit")
    if len(name) >= 3:
        for key, variants in org_mapping.items():
            for variant in variants:
                if name in variant:  # name là substring của variant
                    # Nếu key ĐÃ CÓ prefix (chứa " - ") thì trả về nguyên key
                    if ' - ' in key:
                        return key
                    # Nếu key CHƯA CÓ prefix thì thêm prefix tùy loại
                    if key in organizations:
                        return f'tổ chức - {key}'
                    else:
                        return f'trường - {key}'
    
    # Bước 5: Word overlap match (≥60%)
    name_words = set(name.split())
    if len(name_words) > 0:
        for key, variants in org_mapping.items():
            for variant in variants:
                variant_words = set(variant.split())
                if len(variant_words) == 0:
                    continue
                overlap = len(variant_words & name_words)
                # Nếu có ít nhất 60% từ khớp
                if overlap / len(variant_words) >= 0.6:
                    # Nếu key ĐÃ CÓ prefix (chứa " - ") thì trả về nguyên key
                    if ' - ' in key:
                        return key
                    # Nếu key CHƯA CÓ prefix thì thêm prefix tùy loại
                    if key in organizations:
                        return f'tổ chức - {key}'
                    else:
                        return f'trường - {key}'
    
    # KHÔNG TÌM THẤY
    return None


# Tạo UDF cho Spark
find_org_group_udf = udf(find_org_group, StringType())

print("Đã tạo hàm normalize_text() (CẢI TIẾN: clean special chars, KHÔNG xóa prefix)")
print("Đã tạo hàm find_org_group() (CẢI TIẾN: exact/substring ≥4 chars/reverse substring ≥3 chars/word overlap ≥60%)")
print("Đã tạo UDF cho Spark")

## 4. Đọc dữ liệu từ bảng Post_Entity (chỉ entityType = 'ORG')

In [ ]:
# Đọc dữ liệu từ bảng Post_Entity với JOIN Entity để lọc chỉ entityType = 'ORG'
df_post_entity = spark.sql("""
    SELECT 
        pe.postID,
        pe.entityID,
        pe.entityOrder,
        pe.entityName,
        pe.entityNameNormalized,
        pe.created_at,
        pe.updated_at,
        e.entityType
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'ORG'
""")

total_count = df_post_entity.count()
print(f"✓ Đã đọc {total_count:,} records từ Post_Entity (chỉ entityType = 'ORG')")
df_post_entity.show(10, truncate=False)

## 5. Tạo UDF và áp dụng mapping

In [ ]:
# Tạo UDF và áp dụng mapping
from pyspark.sql.functions import when, lower as spark_lower

norm_udf = udf(normalize_text, StringType())
map_udf = udf(find_org_group, StringType())

df_mapped = df_post_entity \
    .withColumn("lowercased", spark_lower(col("entityName"))) \
    .withColumn("clean", norm_udf(col("lowercased"))) \
    .withColumn("mapped", map_udf(col("clean"))) \
    .withColumn("entityNameNormalized_new", col("mapped"))

print("✓ Đã áp dụng mapping (lưu TẤT CẢ mapped values)")
df_mapped.select("entityName", "lowercased", "clean", "mapped", "entityNameNormalized_new") \
    .show(20, truncate=False)

df_mapped.cache()
df_post_entity.unpersist()
print("✓ Đã cache df_mapped")


## 6. Thống kê kết quả mapping

In [ ]:
# Thống kê kết quả
total = df_mapped.count()
mapped = df_mapped.filter(col("entityNameNormalized_new").isNotNull()).count()
unmapped = total - mapped

print("\n" + "="*70)
print(" KẾT QUẢ MAPPING")
print("="*70)
print(f"Tổng: {total:,} records")
print(f"✓ Mapped: {mapped:,} ({mapped/total*100:.2f}%)")
print(f"✗ Unmapped: {unmapped:,} ({unmapped/total*100:.2f}%)")

# Top nhóm
print("\n TOP 20 NHÓM TRƯỜNG/TỔ CHỨC:")
df_mapped.filter(col("entityNameNormalized_new").isNotNull()) \
    .groupBy("entityNameNormalized_new").count() \
    .orderBy(col("count").desc()) \
    .show(20, truncate=False)

# Top chưa map
if unmapped > 0:
    print("\n TOP 15 CHƯA MAP:")
    df_mapped.filter(col("entityNameNormalized_new").isNull()) \
        .groupBy("entityName").count() \
        .orderBy(col("count").desc()) \
        .show(15, truncate=False)

## 7. Cập nhật cột entityNameNormalized trong bảng Post_Entity

In [ ]:
# Chuẩn bị và thực hiện UPDATE
from pyspark.sql.functions import current_timestamp

df_update = df_mapped.select(
    "postID", "entityID", "entityOrder", "entityName",
    col("entityNameNormalized_new").alias("entityNameNormalized"),
    "created_at", current_timestamp().alias("updated_at")
)

df_update.cache()
count = df_update.count()
print(f"  Updating {count:,} records...")

df_update.createOrReplaceTempView("temp_updates")

spark.sql("""
    MERGE INTO nessie.gold_result_model_multi_task.Post_Entity AS t
    USING temp_updates AS s
    ON t.postID = s.postID AND t.entityID = s.entityID AND t.entityOrder = s.entityOrder
    WHEN MATCHED THEN UPDATE SET
        t.entityNameNormalized = s.entityNameNormalized,
        t.updated_at = s.updated_at
""")

print(" Update completed!")

spark.catalog.dropTempView("temp_updates")
df_update.unpersist()
df_mapped.unpersist()
print("✓ Cleanup done")

## 8. Xác minh kết quả sau khi update

In [ ]:
# Đọc lại dữ liệu sau khi update để xác minhs
df_verify = spark.sql("""
    SELECT 
        pe.entityNameNormalized,
        COUNT(*) as total_records,
        COUNT(DISTINCT pe.entityName) as distinct_original_names
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'ORG'
    GROUP BY pe.entityNameNormalized
    ORDER BY total_records DESC
""")

print("\n" + "="*80)
print("XÁC MINH KẾT QUẢ SAU KHI UPDATE (TOP 30 NHÓM)")
print("="*80)
df_verify.show(30, truncate=False)

total_normalized_groups = df_verify.count()
print(f"\n✓ Tổng số nhóm trường/tổ chức sau khi chuẩn hóa: {total_normalized_groups:,}")

## 9. Dừng Spark Session

In [ ]:
# Cleanup và dừng Spark
import gc

spark.catalog.clearCache()
gc.collect()
spark.stop()

print(" Spark Session stopped!")